<a href="https://colab.research.google.com/github/btgurss/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-15 22:13:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-08-15 22:13:12 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date, count, col
# Read in the Review dataset as a DataFrame


In [ ]:
#Cleaning dataframe
clean = df.dropna()
more_clean = clean.distinct()

In [ ]:
# Create the customers_table DataFrame
customers_df = more_clean.groupby("customer_id").count().withColumnRenamed('count', 'customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40305706|             1|
|    3887504|             4|
|   41739251|             1|
|   28952249|            51|
|   34724923|             1|
|   14225559|             3|
|   26285304|             2|
|     167919|             1|
|   26815475|             9|
|   14908291|             6|
|   27672367|             2|
|   24219298|             4|
|   48356484|            27|
|   18461577|             4|
|   46086404|             1|
|   49996826|             7|
|   20159319|             7|
|   24353265|             2|
|   19833133|             4|
|   51567618|             2|
+-----------+--------------+
only showing top 20 rows



In [ ]:
#Count rows of cleaned dataframe
more_clean.count()

4863497

In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = more_clean.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0545449359|Klutz Fashion For...|
|0615394124|     füdoo Board Kit|
|0615881386|The Revised Crisw...|
|0735328757|Mudpuppy Butterfl...|
|0764955063|The Square of St....|
|0890100039|Kalmbach Building...|
|0954840526|World Prehistoric...|
|0966257545|Ladybug Game NEW ...|
|0972428232| Conversations to Go|
|0980345553|Art of Conversati...|
|0982757751|World of Harmony ...|
|0988179024|All Quiet on the ...|
|1409582485|Listen and Learn ...|
|1554841615|     Cat Smarts Game|
|1556345542|Munchkin 2 - Unna...|
|1567677037|Hot Dots Let’s Le...|
|1572158700|Cat in the Hat Gi...|
|1579823645|MerryMakers Pete ...|
|1585640514|Star Fleet Battle...|
|1589941411|     War of the Ring|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = more_clean.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010KWXLSPF3|   43750961|B004UUHX9M|     247428181| 2014-12-05|
|R100BG8WSTAYPG|   37257103|B00ENKGLHO|       9108602| 2014-04-29|
|R100F00UQBXLAL|   14056891|B004O3CSX6|     579644974| 2011-11-11|
|R1010FUSHO8ASL|   30419449|B000VXNYQI|     496402705| 2012-12-17|
|R101LBRJ366XL1|   36812928|B000JFJC4G|     870047482| 2015-02-28|
|R101LKRSNW6OIK|   28394801|B000FPCWXI|     841597184| 2014-07-06|
|R101TEA798DC3M|   19838846|B0017VO5AW|     484336160| 2015-06-14|
|R101UKS45S1Z0O|   18589384|B000GHWTS8|     632533309| 2009-12-12|
|R101Y71FKFPSEO|   17270128|B004U7JJWE|     366785521| 2013-03-05|
|R102ETLR4XJZFM|   37637960|B000S94NWE|     327618371| 2013-11-08|
|R102IJ4R9UE6QH|   21873089|B00HTMCF0U|     841568092| 2015-02-16|
|R1038ZA60A3YYJ|    2395887|B004S8F7QM|     829220659| 2014-11

In [ ]:
# Create the vine_table. DataFrame
vine_df = more_clean.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010KWXLSPF3|          5|            0|          0|   N|                Y|
|R100BG8WSTAYPG|          5|            1|          1|   N|                Y|
|R100F00UQBXLAL|          5|            0|          0|   N|                N|
|R1010FUSHO8ASL|          5|            0|          1|   N|                Y|
|R101LBRJ366XL1|          5|            0|          0|   N|                Y|
|R101LKRSNW6OIK|          5|            0|          0|   N|                Y|
|R101TEA798DC3M|          4|            1|          1|   N|                Y|
|R101UKS45S1Z0O|          5|            0|          0|   N|                Y|
|R101Y71FKFPSEO|          5|            0|          0|   N|                Y|
|R102ETLR4XJZFM|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cgwk4hayr7zo.us-east-2.rds.amazonaws.com:5432/Module16-Challenge"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)